In [ ]:
import time

notebook_start_time = time.time()

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

root_dir = str(Path().absolute().parent)
if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

In [ ]:
import warnings

warnings.filterwarnings("ignore")

from loguru import logger

from recsys.hopsworks_integration import (
    feature_store,
    llm_ranking_serving,
    two_tower_serving,
)

# Deploying and testing the LLM ranker inference pipeline

In [ ]:
project, fs = feature_store.get_feature_store()

## Deploying the ranking inference pipeline

In [ ]:
ranking_model = llm_ranking_serving.HopsworksLLMRankingModel()
ranking_model.register(project.get_model_registry())

In [ ]:
ranking_deployment = (
    llm_ranking_serving.HopsworksLLMRankingModel.deploy()
)

In [ ]:
ranking_deployment.start()

## Test the ranking inference pipeline

In [ ]:
def get_top_recommendations(ranked_candidates, k=3):
    return [candidate[-1] for candidate in ranked_candidates["ranking"][:k]]

In [ ]:
test_ranking_input = [
        {
            "customer_id": "d327d0ad9e30085a436933dfbb7f77cf42e38447993a078ed35d93e3fd350ecf",
            "month_sin": 1.2246467991473532e-16,
            "query_emb": [
                0.214135289,
                0.571055949,
                0.330709577,
                -0.225899458,
                -0.308674961,
                -0.0115124583,
                0.0730511621,
                -0.495835781,
                0.625569344,
                -0.0438038409,
                0.263472944,
                -0.58485353,
                -0.307070434,
                0.0414443575,
                -0.321789205,
                0.966559,
            ],
            "month_cos": -1.0,
        }
    ]

# Test ranking deployment
ranked_candidates = ranking_deployment.predict(inputs=test_ranking_input)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates["predictions"], k=3)
recommendations

In [ ]:
query_model_deployment = (
    two_tower_serving.HopsworksQueryModel.deploy(ranking_model_type="llmranking")
)

In [ ]:
query_model_deployment.start()

In [ ]:
data = [
    {
        "customer_id": "d327d0ad9e30085a436933dfbb7f77cf42e38447993a078ed35d93e3fd350ecf",
        "transaction_date": "2022-11-15T12:16:25.330916",
    }
]

In [ ]:
ranked_candidates = query_model_deployment.predict(inputs=data)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates["predictions"], k=3)
recommendations

In [ ]:
ranking_deployment.stop()
query_model_deployment.stop()

In [ ]:
notebook_end_time = time.time()
notebook_execution_time = notebook_end_time - notebook_start_time

logger.info(
    f"⌛️ Notebook Execution time: {notebook_execution_time:.2f} seconds ~ {notebook_execution_time / 60:.2f} minutes"
)